### 0. settings

In [164]:
from selenium import webdriver as wd
from bs4 import BeautifulSoup as bs
import time
from IPython.display import display
from IPython.display import clear_output
import random

In [185]:
browser = webdriver.Chrome('./chromedriver')

In [144]:
def progressBar(curr, total):
    percentage = curr/total
    min_scale = 0.02
    total_unit = 50
    arrow_val = 1
    bar_val = int(50*percentage)-1
    dot_val = total_unit - arrow_val - bar_val
    bar = '[' + ''.join(['=' for _ in range(bar_val)] + ['>'] + ['.' for _ in range(dot_val)]) + ']'
    whole_bar = '{} -- {}/{}({}%)'.format(bar, curr, total, int((curr/total)*100))
    return whole_bar

### 1. get all stock id

In [66]:
url = 'https://www.twse.com.tw/zh/page/trading/exchange/STOCK_DAY.html'
browser.get(url)
browser.find_element_by_css_selector('form a').click()

In [67]:
browser.find_element_by_css_selector('.zh.production .view.v1 .category.stk a[data-val="24"]').click()

In [68]:
soup = bs(browser.page_source, 'html.parser')
rlt = soup.select('.zh.production')[0].select('.view.v2')[0].select('.items')[0].select('a')
stock_ids = [i.text.split()[0] for i in rlt]

### 2. crawl single stock

In [186]:
year_range = range(2013, 2019)
month_range = range(1, 13)

browser.get(url)
stock_data = {}

In [187]:
for s_id, curr in enumerate(stock_ids):
    stock_data[curr] = {}
    
    browser.find_element_by_class_name('stock-code-autocomplete').send_keys(curr)
    
    for y_id, yr in enumerate(year_range):
        
        stock_data[curr][yr] = {}
        
        browser.find_element_by_css_selector('#d1 select[name="yy"] option[value="{}"]'.format(yr)).click()
        
        for m_id, mn in enumerate(month_range):  
            
            ###
            clear_output(wait=True)
            display('stock {} -- {}'.format(curr, progressBar(s_id+1, len(stock_ids))),
                    'year  {} -- {}'.format(yr, progressBar(y_id+1, len(year_range))),
                    'month {}   -- {}'.format(mn, progressBar(m_id+1, len(month_range))))            
            ###
            
            
            stock_data[curr][yr][mn] = []
            
            browser.find_element_by_css_selector('#d1 select[name="mm"] option[value="{}"]'.format(mn)).click()
            browser.find_element_by_css_selector('.main.ajax .button.search').click()
            
            time.sleep(random.uniform(1.0, 5.0))
            
            soup = bs(browser.page_source, 'html.parser')
            
            stock_data[curr][yr][mn] = [[td.text for td in tr.select('td')] for tr in soup.select('.data-table')[0].select('tbody tr')]
            
    break

'stock 2302 -- [>..................................................] -- 1/90(1%)'

'year  2018 -- [=================================================>] -- 6/6(100%)'

'month 12   -- [=================================================>] -- 12/12(100%)'

In [194]:
import msgpack

In [196]:
with open('2302.pkl', 'wb') as f:
    msgpack.dump(stock_data['2302'], f)